In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interactive, FloatSlider

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score, plot_roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from joblib import dump, load
import os

import eda

import psycopg2 as pg
import psycopg2.extras as pgex
import sql_fish

RANDOM = 42
connection = sql_fish.fishing_database()

In [3]:
query = "SELECT is_fishing, COUNT(*) as record_count FROM fishingfeatures GROUP BY is_fishing ORDER BY is_fishing;"

pd.io.sql.read_sql(query, connection)

,is_fishing,record_count
0,-1.000000,28027543
1,0.000000,295979
2,0.166667,12
3,0.250000,670
4,0.333333,4096
5,0.400000,9
6,0.666667,4806
7,0.750000,752
8,0.800000,33
9,1.000000,247498


In [4]:
query = "SELECT gear_type, COUNT(*) AS record_count FROM fishingfeatures GROUP BY gear_type ORDER BY record_count DESC;"

gear_record_summary_df = pd.io.sql.read_sql(query, connection)

gear_record_summary_df

,gear_type,record_count
0,drifting_longlines,13968727
1,unknown,6811552
2,trawlers,4369101
3,fixed_gear,1559137
4,purse_seines,1545323
5,trollers,166243
6,pole_and_line,161315


In [5]:
query = "SELECT gear_type, COUNT(DISTINCT mmsi) AS ship_count FROM fishingfeatures GROUP BY gear_type ORDER BY ship_count DESC;"

gear_ship_summary_df = pd.io.sql.read_sql(query, connection)

gear_ship_summary_df

,gear_type,ship_count
0,unknown,120
1,drifting_longlines,110
2,trawlers,49
3,fixed_gear,36
4,purse_seines,28
5,pole_and_line,6
6,trollers,5


In [ ]:
query = "SELECT * FROM fishingfeatures WHERE is_fishing != -1;"

fishing_raw_df = pd.io.sql.read_sql(query, connection)

fishing_raw_df

In [2]:
query = """
        SELECT index, timestamp, is_new_mmsi
        FROM fishingfeatures;
        """

fishing_time_taken_df = pd.io.sql.read_sql(query, connection)

fishing_time_taken_df

,index,timestamp,is_new_mmsi
0,6702,2014-04-12 12:08:21,False
1,6703,2014-04-12 13:41:52,False
2,6704,2014-04-12 14:21:06,False
3,6705,2014-04-12 16:05:04,False
4,6706,2014-04-12 16:27:10,False
...,...,...,...
28581393,28581393,2016-11-23 11:43:42,False
28581394,28581394,2016-11-23 12:11:01,False
28581395,28581395,2016-11-23 13:09:03,False
28581396,28581396,2016-11-23 13:57:02,False


In [31]:
fishing_time_taken_df['time_taken'].value_counts()

0 days 00:00:01.000000000    1533016
0 days 00:00:10.000000000     900974
0 days 00:00:09.000000000     715097
0 days 00:00:02.000000000     645432
0 days 00:00:20.000000000     497393
                              ...   
2 days 05:48:36.000000000          1
2 days 17:53:23.000000000          1
2 days 09:06:36.000000000          1
0 days 18:59:34.000000000          1
4 days 23:41:16.000000000          1
Name: time_taken, Length: 98325, dtype: int64

In [20]:
test_str = fishing_time_taken_df['time_taken'].loc[0]

test_str

In [7]:
int(test_str.split(" days ")[0])

99

In [ ]:
test_str.split(" days ")[1].split(":")

In [6]:
timefactor = [3600, 60, 1]

In [13]:
test_str.split(" days ")[1].split(":")

['15', '16', '53.000000000']

In [30]:
time = convert_strdelta_to_seconds(test_str)

time

0

In [4]:
def convert_strdelta_to_seconds(string):
    
    timefactor = [3600, 60, 1]

    if string == None:
        return 0
    else:
        day_time_split = string.split(" days ")
        return int(day_time_split[0]) * 24 * 3600 + int(sum([float(elem) * factor for elem, factor in zip(day_time_split[1].split(":"), timefactor)]))


In [5]:
fishing_time_taken_df['sec_taken'] = fishing_time_taken_df['time_taken'].apply(convert_strdelta_to_seconds)

In [6]:
fishing_time_taken_df

,index,time_taken,sec_taken
0,6702,0 days 04:15:27.000000000,15327
1,6703,0 days 01:33:31.000000000,5611
2,6704,0 days 00:39:14.000000000,2354
3,6705,0 days 01:43:58.000000000,6238
4,6706,0 days 00:22:06.000000000,1326
...,...,...,...
28581393,28581393,0 days 00:19:01.000000000,1141
28581394,28581394,0 days 00:27:19.000000000,1639
28581395,28581395,0 days 00:58:02.000000000,3482
28581396,28581396,0 days 00:47:59.000000000,2879


In [31]:
import dask

In [8]:
fishing_time_taken_df[['index', 'sec_taken']].to_csv("./data/fishing boats/fishing_sec.csv", index=False)

In [51]:
cursor.execute('rollback;')

In [ ]:
cursor.execute('commit;')